In [ ]:
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image

In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
from glob import glob

In [ ]:
train_path = 'car-damage-dataset/data1a/training'
test_path = 'car-damage-dataset/data1a/validation'

In [ ]:
image_files = glob(train_path + '/*/*.jp*g')
test_files = glob(test_path + '/*/*.jp*g')

In [ ]:
plt.imshow(image.load_img(np.random.choice(image_files)))
plt.show()

## Baseline VGG19 pretrained model

In [ ]:
vgg = VGG19(include_top=False,input_shape=[224,224,3])

In [ ]:
for layer in vgg.layers:
    layer.trainable = False

In [ ]:
x = Flatten()(vgg.output)
x = Dense(64,activation='relu')(x)
x = Dropout(0.2)(x)
prediction = Dense(2,activation='softmax')(x)

In [ ]:
model = Model(inputs=vgg.input, outputs=prediction)

In [ ]:
model.summary()

In [ ]:
model.compile('adam','categorical_crossentropy',metrics=['accuracy'])


In [ ]:
gen = ImageDataGenerator(rescale=1./255)

In [ ]:
test_gen = gen.flow_from_directory(test_path, target_size=(224,224))
print(test_gen.class_indices)
labels = [None]*len(test_gen.class_indices)
for k,v in test_gen.class_indices.items():
    labels[v] = k

In [ ]:
for x,y in test_gen:
    print("min:",x[0].min(),"max",x[0].max())
    plt.title([labels[np.argmax(y[0])]])
    plt.imshow(x[0])
    plt.show()
    break

In [ ]:
train_gen = gen.flow_from_directory(train_path, target_size=(224,224))

In [ ]:
batch_size = 128

r = model.fit_generator(train_gen,validation_data=test_gen,
                        epochs=20, steps_per_epoch=len(image_files)//batch_size,
                        validation_steps=len(test_files)//batch_size)

In [ ]:
model.evaluate(test_gen)

In [ ]:
model.evaluate(train_gen)

In [ ]:
def get_confusion_matrix(data_path, N, model):
    print('generating confusion_matrix')
    predictions = []
    targets = []
    for x,y in gen.flow_from_directory(data_path, target_size=(224,224),shuffle=False):
        p = model.predict(x)
        p = np.argmax(p, axis = 1)
        y = np.argmax(y, axis = 1)
        predictions = np.concatenate((predictions,p))
        targets = np.concatenate((targets, y))
        if len(targets)>=N:
            break
    cm = confusion_matrix(targets, predictions)
    return cm

In [ ]:
cm = get_confusion_matrix(train_path, len(image_files), model)
print(cm)

In [ ]:
valid_cm = get_confusion_matrix(test_path, len(test_files), model)
print(valid_cm)

In [ ]:
plt.plot(r.history['loss'],label = 'train loss')
plt.plot(r.history['val_loss'],label = 'valid loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(r.history['accuracy'], label = 'train accuracy')
plt.plot(r.history['val_accuracy'], label ='validation accuracy')
plt.legend()
plt.show()

## ResNeXt pretrained model

In [ ]:
from classification_models.tfkeras import Classifiers

ResNeXt50, preprocess_input = Classifiers.get('resnext50')
resnext = ResNeXt50(include_top = False, input_shape=(224, 224, 3), weights='imagenet')

In [ ]:
Classifiers.models_names()

In [ ]:
for layer in resnext.layers:
    layer.trainable = False

In [ ]:
x = Flatten()(resnext.output)
x = Dense(64,activation='relu')(x)
x = Dropout(0.2)(x)
prediction = Dense(2,activation='softmax')(x)

In [ ]:
model_resnext = Model(inputs=resnext.input, outputs=prediction)

In [ ]:
model_resnext.compile('adam','categorical_crossentropy',metrics=['accuracy'])

In [ ]:
gen = ImageDataGenerator(rescale=1./255)

In [ ]:
test_gen = gen.flow_from_directory(test_path, target_size=(224,224))
print(test_gen.class_indices)
labels = [None]*len(test_gen.class_indices)
for k,v in test_gen.class_indices.items():
    labels[v] = k

In [ ]:
for x,y in test_gen:
    print("min:",x[0].min(),"max",x[0].max())
    plt.title([labels[np.argmax(y[0])]])
    plt.imshow(x[0])
    plt.show()
    break

In [ ]:
train_gen = gen.flow_from_directory(train_path, target_size=(224,224))

In [ ]:


r_resnext = model_resnext.fit(train_gen,validation_data=test_gen,
                        epochs=20, steps_per_epoch=len(image_files)//batch_size,
                        validation_steps=len(test_files)//batch_size)

In [ ]:
model_resnext.evaluate(train_gen)

In [ ]:
model_resnext.evaluate(test_gen)

In [ ]:
cm = get_confusion_matrix(train_path, len(image_files), model_resnext)
print(cm)

In [ ]:
valid_cm = get_confusion_matrix(test_path, len(test_files), model_resnext)
print(valid_cm)

In [ ]:
plt.plot(r_resnext.history['loss'],label = 'train loss')
plt.plot(r_resnext.history['val_loss'],label = 'valid loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(r_resnext.history['accuracy'], label = 'train accuracy')
plt.plot(r_resnext.history['val_accuracy'], label ='validation accuracy')
plt.legend()
plt.show()

In [ ]:
model_resnext.summary